In [1]:
#export
from exp.nb_12a import *

In [2]:
from fastai.test_utils import *

In [3]:
show_install(True)



```text
=== Software === 
python        : 3.9.6
fastai        : 2.5.0
fastcore      : 1.3.26
fastprogress  : 0.2.7
torch         : 1.9.0
nvidia driver : 471.41
torch cuda    : 10.2 / is available
torch cudnn   : 7605 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : NVIDIA GeForce GTX 970

=== Environment === 
platform      : Linux-5.10.43.3-microsoft-standard-WSL2-x86_64-with-glibc2.31
distro        : #1 SMP Wed Jun 16 23:47:55 UTC 2021
conda env     : fastbook
python        : /home/sandmann/anaconda3/envs/fastbook/bin/python
sys.path      : /home/sandmann/repo/fastai-course-v3/nbs/dl2/selfmade
/home/sandmann/anaconda3/envs/fastbook/lib/python39.zip
/home/sandmann/anaconda3/envs/fastbook/lib/python3.9
/home/sandmann/anaconda3/envs/fastbook/lib/python3.9/lib-dynload

/home/sandmann/anaconda3/envs/fastbook/lib/python3.9/site-packages
/home/sandmann/anaconda3/envs/fastbook/lib/python3.9/site-packages/IPython/extensions
/home/sandmann/.ipython

Tue Aug 

# Data

In [4]:
path = untar_data(URLs.IMDB)

In [5]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))

In [6]:
bs,bptt = 64, 70
data = lm_databunchify(ll, bs, bptt)

In [7]:
vocab = ll.train.proc_x[1].vocab

# Finetunig the LM

In [6]:
# ! wget http://files.fast.ai/models/wt103_tiny.tgz -P {path}

In [7]:
# ! tar xf {path}/wt103_tiny.tgz -C {path}

In [8]:
(path/'pretrained').ls()

[Path('/home/sandmann/.fastai/data/imdb/pretrained/vocab.pkl'),
 Path('/home/sandmann/.fastai/data/imdb/pretrained/pretrained.pth')]

In [9]:
dps = tensor([0.1, 0.15, 0.25, 0.02, 0.2]) * 0.5
tok_pad = vocab.index(PAD)

In [10]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, *dps)

In [11]:
old_wgts = torch.load(path/'pretrained'/'pretrained.pth')
old_vocab = pickle.load(open(path/'pretrained'/'vocab.pkl', 'rb'))

In [12]:
idx_house_new, idx_house_old = vocab.index('house'), old_vocab.index('house')

In [13]:
idx_house_new, idx_house_old

(348, 230)

In [14]:
old_wgts.keys()

odict_keys(['0.emb.weight', '0.emb_dp.emb.weight', '0.rnns.0.weight_hh_l0_raw', '0.rnns.0.module.weight_ih_l0', '0.rnns.0.module.weight_hh_l0', '0.rnns.0.module.bias_ih_l0', '0.rnns.0.module.bias_hh_l0', '0.rnns.1.weight_hh_l0_raw', '0.rnns.1.module.weight_ih_l0', '0.rnns.1.module.weight_hh_l0', '0.rnns.1.module.bias_ih_l0', '0.rnns.1.module.bias_hh_l0', '1.decoder.weight', '1.decoder.bias'])

In [15]:
house_wgt = old_wgts['0.emb.weight'][idx_house_old]
house_bias = old_wgts['1.decoder.bias'][idx_house_old]

In [16]:
house_wgt.shape, house_bias

(torch.Size([300]), tensor(0.1079, device='cuda:0'))

In [17]:
old_wgts['0.emb.weight'].shape

torch.Size([60002, 300])

In [18]:
old_wgts['1.decoder.bias'].shape

torch.Size([60002])

In [19]:
def match_embeds(old_wgts, old_vocab, new_vocab):
    wgts = old_wgts['0.emb.weight']
    bias = old_wgts['1.decoder.bias']
    wgts_m, bias_m = wgts.mean(dim=0), bias.mean()
    new_wgts = wgts.new_zeros(len(new_vocab), wgts.size(1))
    new_bias = bias.new_zeros(len(new_vocab))
    otoi = {v:k for k,v in enumerate(old_vocab)}
    for i,w in enumerate(new_vocab):
        if w in otoi:
            idx = otoi[w]
            new_wgts[i], new_bias[i] = wgts[idx], bias[idx]
        else:
            new_wgts[i], new_bias[i] = wgts_m, bias_m
    
    old_wgts['0.emb.weight'] = new_wgts
    old_wgts['0.emb_dp.emb.weight'] = new_wgts
    old_wgts['1.decoder.weight'] = new_wgts
    old_wgts['1.decoder.bias'] = new_bias
    return old_wgts

In [20]:
wgts = match_embeds(old_wgts, old_vocab, vocab)

In [21]:
test_near(wgts['0.emb.weight'][idx_house_new], house_wgt)
test_near(wgts['1.decoder.bias'][idx_house_new], house_bias)

In [22]:
model.load_state_dict(wgts)

<All keys matched successfully>

In [23]:
model

SequentialRNN(
  (0): AWD_LSTM(
    (emb): Embedding(60003, 300, padding_idx=1)
    (emb_dp): EmbeddingDropout(
      (emb): Embedding(60003, 300, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (output_dp): RNNDropout()
    (decoder): Linear(in_features=300, out_features=60003, bias=True)
  )
)

In [24]:
len(model[0].rnns)

2

In [25]:
def lm_splitter(m):
    groups = []
    for i in range(len(m[0].rnns)): groups.append(nn.Sequential(m[0].rnns[i], m[0].hidden_dps[i]))
    groups += [nn.Sequential(m[0].emb, m[0].emb_dp, m[0].input_dp, m[1])]
    return [list(o.parameters()) for o in groups]

In [26]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(False)

In [27]:
cbs = [partial(AvgStatsCallback, accuracy_flat),
      CudaCallback, Recorder, ProgressCallback,
      partial(RNNTrainer, α=2., β=1.),
      partial(GradientClipping, 0.1)]

In [28]:
learn = Learner(model, data, cross_entropy_flat, adam_opt(), cb_funcs=cbs, splitter=lm_splitter)

In [29]:
lr = 2e-2
cbsched = sched_1cycle(lr, pct_start=0.5, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

In [ ]:
for rnn in model[0].rnns:
    for p in rnn.parameters(): p.requires_grad_(True)

In [ ]:
learn.fit(10, cbs=cbsched)

In [ ]:
torch.save(learn.model[0].state_dict(), path/'finetuned_enc.pth')

In [ ]:
pickle.dump(vocab, open(path/'vocab_lm.pkl', 'wb'))

In [ ]:
torch.save(learn.model.state_dict(), path/'finetuned.pth')

# Classifier

In [ ]:
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))
proc_tok, proc_num, proc_cat = TokenizeProcessor(), NumericalizeProcessor(vocab), CategoryProcessor()

In [ ]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, [proc_tok, proc_num], proc_cat)

In [ ]:
pickle.dump(ll, open(path/'ll_clas.pkl', 'wb'))

In [ ]:
ll = pickle.load(open(path/'ll_clas.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [ ]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

# Ignore Padding

In [8]:
#export
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [9]:
x,y = next(iter(data.train_dl))

In [10]:
x.size()

torch.Size([64, 70])

In [11]:
x

tensor([[    2,    18,    25,  ...,   239,   189,    13],
        [   17,     9,     7,  ...,   102,     9,     7],
        [   16,  3005,    23,  ...,     8,    81,  1739],
        ...,
        [    8,   849,  1480,  ...,     3,     2,    18],
        [  633,    22, 16201,  ...,    64,    83,   155],
        [  345,    26,    21,  ...,   937,  4282,    57]])

In [12]:
lengths = x.size(1) - (x == 1).sum(1)
lengths

tensor([70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
        70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
        70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
        70, 70, 70, 70, 70, 70, 70, 70, 70, 70])

In [13]:
tst_emb = nn.Embedding(len(vocab), 300)

In [14]:
tst_emb(x).shape

torch.Size([64, 70, 300])

In [15]:
64*70

4480

In [16]:
packed = pack_padded_sequence(tst_emb(x), lengths, batch_first=True)

In [17]:
packed

PackedSequence(data=tensor([[ 2.7952,  0.7422,  0.5680,  ..., -0.1471, -1.8697,  0.4242],
        [ 0.7417, -0.5874, -2.2011,  ..., -0.1187, -0.4848,  0.6360],
        [ 0.0952,  0.5742, -1.9913,  ...,  0.1504, -1.0225,  0.7398],
        ...,
        [-0.7464, -0.4574, -0.7304,  ..., -0.6532,  0.9528,  0.1058],
        [-0.6127, -0.4057,  0.5029,  ...,  0.9360,  0.2939, -0.4825],
        [-1.7341,  0.0349, -0.6919,  ..., -0.9535,  1.6054,  1.7652]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64]), sorted_indices=None, unsorted_indices=None)

In [18]:
packed.data.shape

torch.Size([4480, 300])

In [19]:
len(packed.batch_sizes)

70

In [20]:
4480 // 70

64

In [21]:
tst = nn.LSTM(300, 300, 2)

In [22]:
y,h = tst(packed)

In [23]:
unpack = pad_packed_sequence(y, batch_first=True)

In [24]:
unpack[0].shape, unpack[1]

(torch.Size([64, 70, 300]),
 tensor([70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
         70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
         70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
         70, 70, 70, 70, 70, 70, 70, 70, 70, 70]))

In [35]:
#export
class AWD_LSTM1(nn.Module):
    initrange = 0.1
    
    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers,self.pad_token = 1,emb_sz,n_hid,n_layers,pad_token
        self.emb = nn.Embedding(len(vocab), emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l!= n_layers - 1 else emb_sz), 1, batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for _ in range(n_layers)])
        
    def forward(self, input):
        bs,sl = input.size()
        mask = (input == self.pad_token)
        lengths = sl - mask.long().sum(1)
        n_empty = (lengths == 0).sum()
        if n_empty > 0:
            input = input[:-n_empty]
            lengths = lengths[:-n_empty]
            self.hidden = [(h[0][:,:input.size(0)], h[1][:,:input.size(0)]) for h in self.hidden]
        
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output = pack_padded_sequence(raw_output, lengths, batch_first=True)
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            raw_output = pad_packed_sequence(raw_output, batch_first=True)[0]
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
            new_hidden.append(new_h)
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs, mask
    
    def _one_hidden(self, l):
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()
    
    def reset(self):
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

# Concat Pooling

In [41]:
class Pooling(nn.Module):
    def forward(self, input):
        raw_outputs, outputs, mask = input
        output = outputs[-1]
        lengths = output.size(1) - mask.long().sum(dim=1)
        
        avg_pool = output.masked_fill(mask[:,:,None], 0).sum(dim=1)
        avg_pool.div_(lengths.type(avg_pool.dtype)[:,None])
        max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
        x = torch.cat([output[torch.arange(0, output.size(0)), lengths-1], max_pool, avg_pool], dim=1)
        return output, x

In [42]:
emb_sz, nh, nl = 300, 300, 2
tok_pad = vocab.index(PAD)

In [43]:
enc = AWD_LSTM1(len(vocab), emb_sz, nh, nl, tok_pad)
pool = Pooling()

In [44]:
enc.bs = bs
enc.reset()

In [45]:
x,y = next(iter(data.train_dl))
output, c = pool(enc(x))

In [48]:
x.shape

torch.Size([64, 70])

In [47]:
output.shape

torch.Size([64, 70, 300])

In [49]:
test_near((output.sum(dim=2) == 0).float(), (x == tok_pad).float())

In [56]:
c.shape

torch.Size([64, 900])

In [57]:
for i in range(bs):
    length = x.size(1) - (x[i] == 1).long().sum()
    out_unpad = output[i, :length]
    test_near(out_unpad[-1], c[i,:300])
    test_near(out_unpad.max(0)[0], c[i,300:600])
    test_near(out_unpad.mean(0), c[i,600:])

In [59]:
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

In [61]:
class PoolingLinearClassifier(nn.Module):
    
    def __init__(self, layers, drops):
        super().__init__()
        mod_layers = []
        activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]
        for n_in, n_out, p, actn in zip(layers[:-1], layers[1:], drops, activs):
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
        self.layers = nn.Sequential(*mod_layers)
        
    def forward(self, input):
        raw_outputs, output, mask = input
        output = outputs[-1]
        lengths = output.size(1) - mask.long().sum(dim=1)
        
        avg_pool = output.masked_fill(mask[:,:,None], 0).sum(dim=1)
        avg_pool.div_(lengths.type(avg_pool.dtype)[:,None])
        max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
        
        x = torch.cat([output[torch.arange(0, output.size(0)), lengths-1], max_pool, avg_pool], dim=1)
        x = self.layers(x)
        return x

In [63]:
def pad_tensor(t, bs, val=0.):
    if t.size(0) < bs:
        return torch.cat([t, val + t.new_zeros(bs-t.size(0), *t.shape[1:])])
    return t

In [64]:
class SentenceEncoder(nn.Module):
    def __init__(self, module, bptt, pad_idx=1):
        super().__init__()
        self.module,self.bptt,self.pad_idx = module,bptt,pad_idx
        
    def concat(self, arrs, bs):
        return [torch.cat([pad_tensor(l[si],bs) for l in arrs], dim=1) for si in range(len(arrs[0]))]
    
    def forward(self, input):
        bs,sl = input.size()
        self.module.bs = bs
        self.module.reset()
        raw_outputs,outputs,mask = [],[],[]
        for i in range(0, sl, self.bptt):
            r,o,m = self.module(input[:, i: min(i + self.bptt, sl)])
            masks.append(pad_tensor(m, bs, 1))
            raw_outputs.append(r)
            outputs.append(o)
        return self.concat(raw_outputs, bs), self.concat(outputs, bs), torch.cat(masks, dim=1)

In [67]:
def get_text_classifier(vocab_sz, emb_sz, n_hid, n_layers, n_out, pad_token, bptt, output_p=0.4, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5,
                       layers=None, drops=None):
    rnn_enc = AWD_LSTM1(vocab_sz, emb_sz, n_hid, n_layers, pad_token, hidden_p, input_p, embed_p, weight_p)
    enc = SentenceEncoder(rnn_enc, bptt)
    if layers is None: layers = [50]
    if drops is None: drops = [0.1] * len(layers)
    layers = [3 * emb_sz] + layers + [n_out]
    drops = [output_p] + drops
    return SequentialRNN(enc, PoolingLinearClassifier(layers, drops))

In [68]:
emb_sz,nh,nl = 300,300,2
dps = tensor([0.4, 0.3, 0.4, 0.05, 0.5]) * 0.25
model = get_text_classifier(len(vocab), emb_sz, nh, nl, 2, 1, bptt, *dps)

# Training

In [69]:
def class_splitter(m):
    enc = m[0].module
    groups = [nn.Sequential(enc.emb, enc.emb_dp, enc.input_db)]
    for i in range(len(enc.rnns)): groups.append(nn.Sequential(enc_rnns[i], enc.hidden_dps[i]))
    groups.append(m[1])
    return [list(o.parameters()) for o in groups]

In [70]:
for p in model[0].parameters(): p.requires_grad_(False)

In [71]:
cbs = [partial(AvgStatsCallback, accuracy_flat),
      CudaCallback, Recorder, ProgressCallback,
      partial(GradientClipping, clip=0.1)]

In [ ]:
model[0].module.load_state_dict(torch.load(path/'finetuned_enc.pth'))

In [ ]:
learn = Learner(model, data, F.cross_entropy, adam_opt(), cb_funcs=cbs, splitter=class_splitter)

In [5]:
lr = 1e-2
cbsched = sched_1cycle(lr, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

In [ ]:
for p in model[0].module.rnns[-1].parameters(): p.requires_grad_(True)

In [6]:
lr = 5e-3
cbsched = sched_1cycle([lr/2., lr/2, lr/2, lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

In [ ]:
for p in model[0].parameters(): p.requires_grad_(True)

In [ ]:
lr = 1e-3
cbsched = sched_1cycle([lr/8, lr/4, lr/2, lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(2, cbsched)

In [ ]:
x,y = next(iter(data.valid_dl))

In [ ]:
pred_batch = learn.model.eval()(x.cuda())

In [ ]:
pred_ind = []
for inp in x:
    length = x.size(1) - (inp == 1).long().sum()
    inp = inp[:length]
    pred_ind.append(learn.model.eval()(inp[None].cuda()))

In [ ]:
test_near(pred_batch, torch.cat(pred_ind))